**Background**

Latent Dirichlet Analysis (LDA) is a method used to model the generative process of creating discrete data such as text corpora. LDA can take a large amount of data and create descriptions of that data. This method reduces the size of the data to these short descriptions while still maintaining the relationships necessary to carry out various inferences.  The algorithm assumes a generative process for the creation of documents in a corpus D using N words. The topic for a particular word, $z_n$, is modeled as Multinomial($\theta$), where $\theta \sim Dir(\alpha)$. Then, each word, $w_n$, is chosen from a multinomial probability that is conditioned on the topic $z_n$, P($w_n | z_n, \beta)$.  (Blei et al., pg. 996)

LDA is most well-known for its application in the analysis of text data. It is used to create topics for documents, classify documents based on these topics and to determine which documents are similar to one another. The algorithm can also be used in other problems that have a similar structure to the document generating method described above. For example, in Blei et.al, 2003, the authors used a data set where web site users provide information about movies they enjoy. In this example, the users are analogous to the documents and their preferred movies are “words”.  Topics can be found by determining similar movies. 

There are multiple alternative algorithms that can be used for text analysis. These include the term-frequency inverse-document-frequency matrix, latent semantic indexing (LSI) and the probabilistic latent semantic indexing (pLSI) model. In Section 7 of the paper, the authors present the results of document modeling and document classification. In this section, the authors’ results show that their implementation of LDA performs better than competing methods in terms of perplexity measure, where better generalization performance is defined by a smaller perplexity measure. 

Perplexity measure is defined as Exp $(-\frac{\sum_{d=1}^M log p(\textbf{w}_d}{\sum_{d=1}^M N_d})$.

 In addition, this section demonstrates that other methods are prone to overfitting. As the number of topic increases, some of the alternative algorithms induce words that have small probabilities. This occurs because the documents in the corpus are divided into more collections. This can result in the perplexity measure becoming very large for these alternative algorithms. The problem comes from the requirement that the topic proportions in a future document must be seen in at least one of the training documents. On the other hand, LDA does not have this overfitting problem. For more detail, see Section 7 of Blei et al.

A disadvantage of LDA is that exact inference of the posterior is impossible as a result of intractability. Thus, it is necessary for the user to implement an approximating technique such as variational inference, a Gibbs sampler, or another technique. 








**References**

1.	David M. Blei, Andrew Y. Ng, and Michael I. Jordan, Latent Dirichlet Allocation, Journal of Machine Learning Research 3, 2003, pg. 993-1022.
2.	Max Sklar, Fast MLE Computation for the Dirichlet Multinomial, May 2014.
3.	F. Maxwell Harper and Joseph A. Konstan. 2015. The MovieLens Datasets: History and Context. ACM Transactions on Interactive Intelligent Systems (TiiS) 5, 4, Article 19 (December 2015), 19 pages. DOI=http://dx.doi.org/10.1145/2827872


In [1]:
! pip install stop_words

You are using pip version 8.0.3, however version 8.1.1 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


In [2]:
import collections

s1 = "The quick brown fox"
s2 = "Brown fox jumps over the jumps jumps jumps"
s3 = "The the the lazy dog elephant."
s4 = "The the the the the dog peacock lion tiger elephant"

docs = collections.OrderedDict()
docs["s1"] = s1
docs["s2"] = s2
docs["s3"] = s3
docs["s4"] = s4 
for k, v in docs.items():
    print(k,v)
    

s1 The quick brown fox
s2 Brown fox jumps over the jumps jumps jumps
s3 The the the lazy dog elephant.
s4 The the the the the dog peacock lion tiger elephant


In [3]:
len(docs)

4

**Function to make Corpus Matrix**

Function returns a list, 0th element is a 3 dimensional array, each element corresponds to a document in the corpus where the columns are the unique words in that document and the rows are the words in that document (stop words removed).  Second element in list is the column names for each document.

In [4]:
import string
import stop_words
import numpy as np
from stop_words import get_stop_words

stopWords = get_stop_words('english')

#Function to make document, word matricies for LDA#
def make_word_matrix(corpus):

    #Define list to store corpus data#
    c = []
    #Define list to store order of words for each document#
    wordOrder = []
    #Define table to remove punctuation
    table = dict.fromkeys(map(ord, string.punctuation))

    M = len(corpus)
    #For each document in docs, caculate frequency of the words#
    for i in corpus:
        #Remove punctuation 
        text = docs[i].translate(table)
        #Splits string by blankspace and goes to lower case#
        words = text.lower().split()

        #Remove stop words#
        text = [word for word in words if word not in stopWords]

        #Find total number of words in each document#
        N = len(text)

        #Find number of unique words in each document#
        Vwords = list(set(text))
        wordOrder.append(Vwords)

    #Find unique words in the corpus, this is the vocabulary#    
    wordOrder = list(set(x for l in wordOrder for x in l))
    wordOrder = sorted(wordOrder)

    #Find the number of unique words in the corpus vocabulary#
    V = len(wordOrder)

    #For each document in docs, caculate frequency of the words#
    for i in corpus:
        #Remove punctuation 
        text = docs[i].translate(table)
        #Splits string by blankspace and goes to lower case#
        words = text.lower().split()

        #Remove stop words#
        text = [word for word in words if word not in stopWords]

        #Find total number of words in each document#
        N = len(text)

        #Create matrix to store words for each document#
        wordsMat = np.zeros((N, V))
        count = 0
        for word in text:
            v = wordOrder.index(word)
            wordsMat[count, v] = 1
            count = count + 1
        c.append(wordsMat)

    return [c, wordOrder, M] 

In [5]:
corpusMatrix = make_word_matrix(docs)
corpusMatrix

[[array([[ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.],
         [ 1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
         [ 0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.]]),
  array([[ 1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
         [ 0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.],
         [ 0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.],
         [ 0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.],
         [ 0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.],
         [ 0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.]]),
  array([[ 0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.],
         [ 0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
         [ 0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.]]),
  array([[ 0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
         [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.],
         [ 0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.],
         [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.],
         [ 0.,  0.

**Variational Inference**

In [6]:
import numpy as np
import scipy
from scipy import special

**E-Step:** This function uses variational inference to perform the E step in the EM algorithm to estimate the paramteters in the model.  The output of this function are the matricies gamma and phi, where gamma (k vector) is the Dirichlet paramteters and the matrix phi (N x k, where k is the number of topics) are the multinomial paramters.  See page 1004 of paper for derivation.

In [7]:
def Estep(k, d, alpha, beta, corpusMatrix, tol):    
    
    #storing the total number of words and the number of unique words
    N = corpusMatrix[0][d].shape[0]
    V = corpusMatrix[0][d].shape[1]
    
    #initialize phi and gamma
    oldPhi  = np.full(shape = (N,k), fill_value = 1/k)
    gamma = alpha + N/k
    newPhi = oldPhi
    converge = 0 
    
    
    count = 0
    
    while converge == 0:
        newPhi  = np.zeros(shape = (N,k))
        for n in range(0, N):
            for i in range(0,k):
                newPhi[n,i] = (beta[i, list(corpusMatrix[0][d][n,:]).index(1)])*np.exp(scipy.special.psi(gamma[i]))
        newPhi = newPhi/np.sum(newPhi, axis = 1)[:, np.newaxis] #normalizing the rows of new phi

        for i in range(0,k):
            gamma[i] = alpha[i] + np.sum(newPhi[:, i]) #updating gamma


        criteria = (1/(N*k)*np.sum((newPhi - oldPhi)**2))**0.5
        if criteria < tol:
            converge = 1
        else:
            oldPhi = newPhi
            count = count +1
            converge = 0
    return (newPhi, gamma)

**Parameter Estimation**

**M Step:** In the E step above, we maximized a lower bound with respect to gamma and phi, and in the M step, for fixed values of these variational parameters, we maximize the lower bound of the log likelihood with repsect to alpha and beta to update these values (combined, these two steps give approximate empirical Bayes estimates for the LDA model).  See pg. 1006 and appendix A.2 for derivation.  

The alphaUpdate() function uses the linear Newton-Rhapson method to update the Dirichlet parameters, alpha, while the Mstep() function maximizes for alpha and beta.

In [8]:
#Update alpha using linear Newton-Rhapson Method#
def alphaUpdate(k, M, alphaOld, gamma, tol):
    h = np.zeros(k)
    g = np.zeros(k)
    alphaNew = np.zeros(k)

    converge = 0
    while converge == 0:
        for i in range(0, k):
            docSum = 0
            for d in range(0, M):
                docSum += scipy.special.psi(gamma[d][i]) - scipy.special.psi(np.sum(gamma[d]))
            g[i] = M*(scipy.special.psi(sum(alphaOld)) - scipy.special.psi(alphaOld[i])) + docSum
            h[i] = M*scipy.special.polygamma(1, alphaOld[i])
        z =  -scipy.special.polygamma(1, np.sum(alphaOld))
        c = np.sum(g/h)/(1/z + np.sum(1/h))
        step = (g - c)/h
        if np.linalg.norm(step) < tol:
            converge = 1
        else:
            converge = 0
            alphaNew = alphaOld + step
            alphaOld = alphaNew

    return alphaNew

In [9]:
def Mstep(k, M, phi, gamma, alphaOld, corpusMatrix, tol):
    #Calculate beta#
    V = corpusMatrix[0][0].shape[1]
    beta = np.zeros(shape = (k,V))

    for i in range(0,k):
        for j in range(0,V):
            wordSum = 0
            for d in range(0,M):
                Nd = corpusMatrix[0][d].shape[0]
                for n in range(0, Nd):
                    wordSum += phi[d][n,i]*corpusMatrix[0][d][n,j]
            beta[i,j] = wordSum
    #Normalize the rows of beta#
    beta = beta/np.sum(beta, axis = 1)[:, np.newaxis]
    
    ##Update ALPHA##
    alphaNew = alphaUpdate(k, M, alphaOld, gamma, tol)
    return(alphaNew, beta)



**LDA Function:**
Finally, we implement the entire Latent Dirichlet Allocation method in the LDA function, which takes as its arguments k (the number of topics), D (the number of documents in the corpus), a corpus matrix (the output from make_word_matrix above) and a tolerance (which sets the convergence criteria for the while loops).  For each document d, the function runs until the alpha or beta parameters converge, by first running the E step and then the M step for each document separately.  The final values of phi, gamma, alpha and beta are returned for all D documents in a list.

In [10]:
#k = number of topics, D = number of documents#
#corpus matrix is output of make_word_matrix# 
def LDA(k, M, corpusMatrix, tol):

    
    output = []
    
    converge = 0
    #initialize alpha and beta for first iteration
    alphaOld = 10*np.random.rand(k)
    V = corpusMatrix[0][0].shape[1]
    betaOld = np.random.rand(k, V)
    betaOld = betaOld/np.sum(betaOld, axis = 1)[:, np.newaxis]
    
    while converge == 0:
        phi = []
        gamma = []
        #looping through the number of documents
        for d in range(0,M): #M is the number of documents
            phiT, gammaT = Estep(k, d, alphaOld, betaOld, corpusMatrix, tol)
            phi.append(phiT)
            gamma.append(gammaT)
            
        alphaNew, betaNew = Mstep(k, M, phi, gamma, alphaOld, corpusMatrix, tol)
    
        if np.linalg.norm(alphaOld - alphaNew) < tol or np.linalg.norm(betaOld - betaNew) < tol:
            converge =1
        else: 
            converge =0
            alphaOld = alphaNew
            betaOld = betaNew
    output.append([phi, gamma, alphaNew, betaNew])
        
    return output

In [11]:
LDA(3, 4, corpusMatrix, 0.1)

[[[array([[ 0.12289737,  0.15669001,  0.72041262],
          [ 0.43619702,  0.19079607,  0.37300691],
          [ 0.00195915,  0.08277618,  0.91526466]]),
   array([[ 0.47978723,  0.21055167,  0.3096611 ],
          [ 0.00252532,  0.10704752,  0.89042716],
          [ 0.56793174,  0.2219656 ,  0.21010266],
          [ 0.56793174,  0.2219656 ,  0.21010266],
          [ 0.56793174,  0.2219656 ,  0.21010266],
          [ 0.56793174,  0.2219656 ,  0.21010266]]),
   array([[ 0.51061311,  0.15562109,  0.3337658 ],
          [ 0.1980239 ,  0.18810243,  0.61387368],
          [ 0.44255284,  0.078629  ,  0.47881815]]),
   array([[ 0.1909399 ,  0.17839313,  0.63066696],
          [ 0.25619717,  0.09972446,  0.64407837],
          [ 0.13248979,  0.0094582 ,  0.858052  ],
          [ 0.55265073,  0.10621655,  0.34113273],
          [ 0.42963913,  0.07508032,  0.49528055]])],
  [array([  6.23114479,   2.97648098,  10.73108617]),
   array([  8.42413077,   3.75168029,  10.76290088]),
   array([  6.82

## Future Steps

- Compare output to Python package
- Test on corpus in paper
- Model topics on different corpus
- Time and optimize (use Cython, quite slow now)
- Run collaborative filtering
- Compare to Gibbs sampling

In [1]:
import nltk


In [3]:
nltk.download("gutenberg")

[nltk_data] Downloading package gutenberg to
[nltk_data]     /Users/annayanchenko/nltk_data...
[nltk_data]   Unzipping corpora/gutenberg.zip.


True

In [6]:
from nltk.corpus import gutenberg
gutenberg.fileids()

['austen-emma.txt',
 'austen-persuasion.txt',
 'austen-sense.txt',
 'bible-kjv.txt',
 'blake-poems.txt',
 'bryant-stories.txt',
 'burgess-busterbrown.txt',
 'carroll-alice.txt',
 'chesterton-ball.txt',
 'chesterton-brown.txt',
 'chesterton-thursday.txt',
 'edgeworth-parents.txt',
 'melville-moby_dick.txt',
 'milton-paradise.txt',
 'shakespeare-caesar.txt',
 'shakespeare-hamlet.txt',
 'shakespeare-macbeth.txt',
 'whitman-leaves.txt']

In [12]:
emma = gutenberg.words('austen-emma.txt')


In [10]:
nltk.download("state_union")

[nltk_data] Downloading package state_union to
[nltk_data]     /Users/annayanchenko/nltk_data...
[nltk_data]   Unzipping corpora/state_union.zip.


True

In [11]:
from nltk.corpus import state_union
state_union.fileids()

['1945-Truman.txt',
 '1946-Truman.txt',
 '1947-Truman.txt',
 '1948-Truman.txt',
 '1949-Truman.txt',
 '1950-Truman.txt',
 '1951-Truman.txt',
 '1953-Eisenhower.txt',
 '1954-Eisenhower.txt',
 '1955-Eisenhower.txt',
 '1956-Eisenhower.txt',
 '1957-Eisenhower.txt',
 '1958-Eisenhower.txt',
 '1959-Eisenhower.txt',
 '1960-Eisenhower.txt',
 '1961-Kennedy.txt',
 '1962-Kennedy.txt',
 '1963-Johnson.txt',
 '1963-Kennedy.txt',
 '1964-Johnson.txt',
 '1965-Johnson-1.txt',
 '1965-Johnson-2.txt',
 '1966-Johnson.txt',
 '1967-Johnson.txt',
 '1968-Johnson.txt',
 '1969-Johnson.txt',
 '1970-Nixon.txt',
 '1971-Nixon.txt',
 '1972-Nixon.txt',
 '1973-Nixon.txt',
 '1974-Nixon.txt',
 '1975-Ford.txt',
 '1976-Ford.txt',
 '1977-Ford.txt',
 '1978-Carter.txt',
 '1979-Carter.txt',
 '1980-Carter.txt',
 '1981-Reagan.txt',
 '1982-Reagan.txt',
 '1983-Reagan.txt',
 '1984-Reagan.txt',
 '1985-Reagan.txt',
 '1986-Reagan.txt',
 '1987-Reagan.txt',
 '1988-Reagan.txt',
 '1989-Bush.txt',
 '1990-Bush.txt',
 '1991-Bush-1.txt',
 '1991-B